### <center>面向对象</center>

In [7]:
ss = set([1,2,1])
ss.add(1)
ss

{1, 2}

In [6]:
tuple([1,2,3,1])  # 不可变

(1, 2, 3, 1)

### Methods

In [1]:
def foo(x):
    print("executing foo(%s)" % (x))


class A(object):
    def foo(self, x):
        print("executing foo(%s,%s)" % (self, x))

    @classmethod
    def class_foo(cls, x):
        print("executing class_foo(%s,%s)" % (cls, x))

    @staticmethod
    def static_foo(x):
        print("executing static_foo(%s)" % x)


a = A()
a.class_foo(7)
a.foo(8)
a.static_foo(9)

A.class_foo(10)
A.static_foo(11)

executing class_foo(<class '__main__.A'>,7)
executing foo(<__main__.A object at 0x7f8b7c8ce0f0>,8)
executing static_foo(9)
executing class_foo(<class '__main__.A'>,10)
executing static_foo(11)


#### 对象转字典

In [1]:
class Student:
    name = ''
    age = 0

    def __init__(self, name, age):
        self._name = name
        self.age = age
        
    def to_json(self):
        return class_to_dict(self)
stu = Student('', 20)

In [3]:
stu.to_json()

{'_name': '', 'age': 20}

In [2]:
def convert_to_dict_list(objs):
    '''把对象列表转换为字典列表'''
    obj_arr = []
    for o in objs:
        #把Object对象转换成Dict对象
        dict = {}
        dict.update(o.__dict__)
        obj_arr.append(dict)
    return obj_arr
convert_to_dict_list([stu, stu])

[{'_name': 'XXX', 'age': 20}, {'_name': 'XXX', 'age': 20}]

In [2]:
def class_to_dict(obj):
    '''把对象(支持单个对象、list、set)转换成字典'''
    is_list = obj.__class__ == [].__class__
    is_set = obj.__class__ == set().__class__
    if is_list or is_set:
        obj_arr = []
        for o in obj:
            #把Object对象转换成Dict对象
            dict = {}
            dict.update(o.__dict__)
            obj_arr.append(dict)
        return obj_arr
    else:
        dict = {}
        dict.update(obj.__dict__)
        return dict
class_to_dict([stu, stu])

[{'_name': '', 'age': 20}, {'_name': '', 'age': 20}]

In [5]:
class_to_dict(stu)

{'name': 'XXX', 'age': 20}

#### 字典转对象

In [6]:
def str2list(string,type=str):
    return list(map(type, string))
str2list('12345',int)

[1, 2, 3, 4, 5]

In [7]:
# 返回对象object的属性和属性值的字典对象，如果没有参数，就打印当前调用位置的属性和属性值 类似 locals()
vars(stu) 

{'name': 'XXX', 'age': 20}

In [8]:
stu.__dict__

{'name': 'XXX', 'age': 20}

In [9]:
import collections
try:
  # Python 2.7+
  basestring
except NameError:
  # Python 3.3+
  basestring = str
    
def objtodict(obj):
  """ 
  Recursively convert a Python object graph to sequences (lists)
  and mappings (dicts) of primitives (bool, int, float, string, ...)
  """
  if isinstance(obj, basestring):
    return obj
  elif isinstance(obj, dict):
    return dict((key, objtodict(val)) for key, val in obj.items())
  elif isinstance(obj, collections.Iterable):
    return [todict(val) for val in obj]
  elif hasattr(obj, '__dict__'):
      # var():如果默认不输入参数，就打印当前调用位置的属性和属性值，相当于locals()的功能。
    return objtodict(vars(obj))  # vars():本函数是实现返回对象object的属性和属性值的字典对象
  elif hasattr(obj, '__slots__'):
    return objtodict(dict((name, getattr(obj, name)) for name in getattr(obj, '__slots__')))
  return obj
objtodict(stu)

{'name': 'XXX', 'age': 20}

### __getitem__()
如果在类中定义了__getitem__()方法，那么他的实例对象（假设为P）就可以这样P[key]取值。
当实例对象做P[key]运算时，就会调用类中的__getitem__()方法。

In [11]:
class DataTest:
    def __init__(self, id, address):
        self.id = id
        self.address = address
        self.d = {"id": self.id,
                  "add": self.address
                  }

    def __getitem__(self, key):
        # return "hello"
        return self.d.get(key, 2)
data = DataTest(1, "192.168.2.11")

In [12]:
data[2]

2

In [13]:
data["add"]

'192.168.2.11'

### object property

In [2]:
class O(object):
    def __init__(self):
        self.o = 1
oo = O()

In [3]:
def prn_obj(obj):
    print ('\n'.join(['%s:%s' % item for item in obj.__dict__.items()]))

In [4]:
prn_obj(oo)

o:1


### slots
作用： 限定一个类创建的实例只能有固定的实例属性(实例变量)

当一个类需要创建大量实例时，可以通过__slots__声明实例所需要的属性
例如，class Foo(object): __slots__ = ['foo']。这样做带来以下优点：
更快的属性访问速度
减少内存消耗

默认情况下，访问一个实例的属性是通过访问该实例的__dict__来实现的。如访问a.x就相当于访问a.__dict__['x']

Python内置的字典本质是一个哈希表，它是一种用空间换时间的数据结构。为了解决冲突的问题，当字典使用量超过2/3时，Python会根据情况进行2-4倍的扩容。
由此可预见，取消__dict__的使用可以大幅减少实例的空间消耗。

In [14]:
class Person(object):
    __slots__ = ("name", "age")

In [15]:
P = Person()

In [16]:
P.score = 100

AttributeError: 'Person' object has no attribute 'score'

### type
类的定义过程，其实是type类型实例化的过程。
和 dict 类似，type 也是一个工厂构造函数，调用其将返回 一个type类型的实例（即 类）。


type 有两个重载版本：
- type(object)，即我们最常用的版本。
- type(name, bases, dict)，一个更强大的版本。通过指定 类名称（name）、父类列表（bases）和 属性字典（dict） 动态合成一个类。

In [17]:
Integer = type( 'Integer', 
               (int, ), 
               {'name': 'my integer',
                'increase': lambda self, num: num + 1    # 很酷的写法，不是么
                }
              )

In [18]:
integer = Integer()
integer.increase(2)

3

### meta_class
？？？

In [31]:
sensitive_words_list = ['asshole', 'fuck', 'shit']
def detect_sensitive_words(string):
    '''检测敏感词汇'''
    words_detected = filter(lambda word: word in string.lower(), sensitive_words_list)
    if words_detected:
        raise NameError('Sensitive words {0} detected in the string "{1}".'
            .format(
                ', '.join(map(lambda s: '"%s"' % s, words_detected)),
                string
            )
        )
# detect_sensitive_words("shitcode")

In [32]:
class CleanerMeta(type):
    def __new__(cls, class_name, bases, attrs):   # 可以通过重载元类的 __new__ 方法，修改 类定义 的行为
        detect_sensitive_words(class_name)  # 检查类名
        map(detect_sensitive_words, attrs.iterkeys())  #检查属性名
        print ("Well done! You are a polite coder!") # 如无异常，输出祝贺消息
        return super(CleanerMeta, cls).__new__(cls, class_name, bases, attrs)
        # 重要！这行一定不能漏！！这回调用内建的类构造器来构造类，否则定义好的类将会变成 None

In [33]:
class APIBase(object):
    __metaclass__ = CleanerMeta   # Integer = IntMeta('Integer', (int, ), {})

In [34]:
class ImAGoodBoy(APIBase):
    a_polite_attribute = 1
imAGoodBoy = ImAGoodBoy()

In [38]:
class PretendToBePolite(APIBase):
    def __fuck_your_asshole(self):
        pass
pretendToBePolite = PretendToBePolite()

In [36]:
class FuckMyBoss(APIBase):
    pass
f = FuckMyBoss()

### eval
eval() 函数用来执行一个字符串表达式，并返回表达式的值。

In [7]:
d = str({
    "1":"2"
})
print(d)
dd = eval(d)
print(dd)

{'1': '2'}
{'1': '2'}


In [10]:
eval("3*7")

21

In [6]:
s = "Hello \n World!"
print(s)
print(repr(s))  # repr() 函数将对象转化为供解释器读取的形式。

Hello 
 World!
'Hello \n World!'


### is和==的比较
- is也被叫做同一性运算符，这个运算符比较判断的是对象间的唯一身份标识，也就是id是否相同
- ==是python标准操作符中的比较操作符，用来比较判断两个对象的value(值)是否相等

In [9]:
float(10) == float(10.0)

True

In [10]:
10 is 10.0

False

In [11]:
10==10.0

True

In [15]:
s2 = 10.0
id(s2)  # id() 内存地址

140615773838552

In [14]:
s1 = 10.0
id(s1)

140615773838432

In [16]:
s1 is s2

False